In [ ]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and augmenting images
from skimage.io import imread
from skimage.transform import rotate, resize

# for splitting train-test set and evaluating the model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# PyTorch libraries 
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

# Make sure the images.zip is in the same directory and same path
# loading dataset
!unzip images.zip

In [3]:
train = pd.read_csv('csv/mask.csv')

# loading training images
train_img = []
train_img_values = []
for img_name in train['image_names']:
    image_path = 'images/' + img_name 
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    train_img.append(img)


train_x = np.array(train_img)

# defining the labels
train_y = train['mask_or_not'].values

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=13, stratify=train_y)
print("Number of images (Before Image Augmentation) in Training set : ",train_x.shape[0],"  Number of images in Testing set : ",test_x.shape[0])

Number of images (Before Image Augmentation) in Training set :  652   Number of images in Testing set :  164


In [4]:
# Image Augmentation
final_train_x = []
final_train_y = []

#train_x.shape[0]
for i in range(train_x.shape[0]):
    final_train_x.append(train_x[i])
    final_train_x.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_x.append(np.fliplr(train_x[i]))
    for j in range(3):
      final_train_y.append(train_y[i])

train_x = np.array(final_train_x)
train_y = np.array(final_train_y)
print("Number of images (After Image Augmentation) in Training set : ",train_x.shape[0],"  Number of images in Testing set : ",test_x.shape[0])

Number of images (After Image Augmentation) in Training set :  1956   Number of images in Testing set :  164


In [5]:
# converting training images and its labels into torch format
train_x = train_x.reshape(1956, 3, 224, 224) #1956 is the number of training images
train_x  = torch.from_numpy(train_x)
train_x = train_x.float()
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


# converting test images and its labels into torch format
test_x = test_x.reshape(164, 3, 224, 224) #164 is the number of test images
test_x  = torch.from_numpy(test_x)
test_x = test_x.float()
test_y = test_y.astype(int)
test_y = torch.from_numpy(test_y)

In [17]:
# Define the model
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
        )

        self.linear_layers = Sequential(
            Linear(128 * 14 * 14, 512),
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,10),
            ReLU(inplace=True),
            Dropout(),
            Linear(10,2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [18]:
model = Net()
optimizer = Adam(model.parameters(), lr=0.000075)
criterion = CrossEntropyLoss()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [ ]:
# batch size of the model
batch_size = 32

# number of epochs to train the model
n_epochs = 13

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    permutation = torch.randperm(train_x.size()[0])
    training_loss = []
    for i in range(0,train_x.size()[0], batch_size):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step() 
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)


In [ ]:
# batch size of the model
batch_size = 64
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in range(0,train_x.size()[0], batch_size):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

In [ ]:
# checking the performance on validation set
torch.manual_seed(0)
# batch size of the model
batch_size = 64
# prediction for test set
prediction = []
target = []
permutation = torch.randperm(test_x.size()[0])
for i in range(0,test_x.size()[0], batch_size):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = test_x[indices], test_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# Test accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('Test accuracy: \t', np.average(accuracy))
torch.save(model, 'CNN-model.pth')

The below code is used to test the model with a specific image.

In [ ]:
model = torch.load('CNN-model.pth')
image_path = 'images/407.jpg' #Specify the image path
img = imread(image_path)
img = img/255
img = resize(img, output_shape=(3, 224, 224),
               mode='constant', anti_aliasing=True)
img = img.astype('float32')
img = np.array(img)

img = img.reshape(1, 3, 224, 224) 
img = torch.from_numpy(img)
img = img.float()

output = model(img.cuda())
softmax = torch.exp(output).cpu()
prob = list(softmax.detach().numpy())
predictions = np.argmax(prob, axis=1)
print(predictions)